# STEAMSPY - DATA CLEANING

#### In this chapter, we are going to read our raw data (details_data.csv), clean and prepare for analysis

In [2]:
# Importing libraries
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sys
print(sys.version)

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]


In [3]:
pd.set_option("display.max_columns", 40)
pd.set_option("display.max_rows", 100)

In [8]:
det_df = pd.read_csv(
    r"..\SteamSpy\details_data.csv"
)
det_df

,Unnamed: 0,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,languages,genre,tags
0,0,570,Dota 2,Valve,Valve,NaN,1675039,360483,0,"200,000,000 .. 500,000,000",37327,1417,902,909,0.0,0.0,0.0,736843,"English, Bulgarian, Czech, Danish, Dutch, Finn...","Action, Free to Play, Strategy","{'Free to Play': 59115, 'MOBA': 19781, 'Multip..."
1,0,1063730,New World,Amazon Games,Amazon Games,NaN,182077,77125,0,"50,000,000 .. 100,000,000",4641,102,1946,150,3999.0,3999.0,0.0,14858,"English, French, Italian, German, Spanish - Sp...","Action, Adventure, Massively Multiplayer, RPG","{'Massively Multiplayer': 693, 'Open World': 6..."
2,0,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",NaN,1269439,943280,0,"50,000,000 .. 100,000,000",24540,774,8582,321,0.0,0.0,0.0,301044,"English, Korean, Simplified Chinese, French, G...","Action, Adventure, Free to Play, Massively Mul...","{'Survival': 14403, 'Shooter': 12238, 'Battle ..."
3,0,1172470,Apex Legends,Respawn Entertainment,Electronic Arts,NaN,568429,137990,0,"50,000,000 .. 100,000,000",8255,617,1279,328,0.0,0.0,0.0,288480,"English, French, Italian, German, Spanish - Sp...","Action, Adventure, Free to Play","{'Free to Play': 1746, 'Multiplayer': 1210, 'B..."
4,0,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,NaN,6612258,833900,0,"50,000,000 .. 100,000,000",29864,593,6805,268,0.0,0.0,0.0,1137059,"English, Czech, Danish, Dutch, Finnish, French...","Action, Free to Play","{'FPS': 89213, 'Shooter': 64124, 'Multiplayer'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63052,0,848400,Simian Rising,All Caps Industries,All Caps Industries,NaN,0,0,0,"0 .. 20,000",0,0,0,0,1299.0,1299.0,0.0,0,English,"Action, Adventure, Indie, Simulation, Strategy...",[]
63053,0,2479580,Prop Game,Olivette Noa,Olivette Noa,NaN,4,0,0,"0 .. 20,000",0,0,0,0,199.0,199.0,0.0,0,"English, French","Casual, Indie","{'Casual': 271, 'Arcade': 245, 'Battle Royale'..."
63054,0,2105370,Mortal Fighter,Fly High Games,Fly High Games,NaN,5,2,0,"0 .. 20,000",0,0,0,0,99.0,99.0,0.0,0,English,Action,"{'Martial Arts': 49, 'Action': 46, 'PvP': 43, ..."
63055,0,1859220,疯狂厨房VR,703 Studio,703 Studio,NaN,0,1,0,"0 .. 20,000",0,0,0,0,799.0,799.0,0.0,0,"English, Not supported, Simplified Chinese","Casual, Simulation","{'Casual': 65, 'VR': 53, 'Simulation': 47, 'Fu..."


#### The following columns don't contain valuable data, thus we will drop them
- "Unnamed: 0" - consits of zeros
- "average_2weeks" - average gametime in 2 weeks
- "median_2weeks" - average gametime in 2 weeks
- "score_rank" and "userscore" - these columns have a lot of missing values so they are not reliable information for evaluation
- "ccu" - peak concurrent users the day before

In [9]:
det_df.columns
det_df.drop(
    ["Unnamed: 0", "average_2weeks", "median_2weeks", "score_rank", "userscore", "ccu"],
    axis=1,
    inplace=True,
)

#### Instead of "score_rank" and "userscore" columns, to evaluate app rating we can use "positive" and "negative" votes to create a rating for the game

In [10]:
total_votes = det_df["positive"] + det_df["negative"]
rate = 100 * det_df["positive"] / total_votes
det_df["rating"] = rate.round(decimals=1)

#### "owners" column has min and max values separated by '..', we will rename column into "avg_owners" and replace values into average owners

In [11]:
# Renaming "owners" column into "avg_owners"
det_df["owners"]
det_df["owners"].isna().any()
det_df.rename(columns={"owners": "avg_owners"}, inplace=True)

In [12]:
# Defining function for calculation
def avg_func(row):
    split_row = row.split(" .. ")
    avg = (int(split_row[0].replace(",", "")) + int(split_row[1].replace(",", ""))) / 2
    return avg

In [13]:
det_df["avg_owners"] = det_df["avg_owners"].apply(avg_func)
det_df["avg_owners"] = det_df["avg_owners"].round().astype(int)

#### "price", "initialprice" and "discount" columns data types are float which is right, however prices should be 1/100 and the currency in USD

In [14]:
det_df.rename(columns={"price": "price_USD"}, inplace=True)
det_df["price_USD"] = det_df["price_USD"] / 100


det_df.rename(columns={"initialprice": "initialprice_USD"}, inplace=True)
det_df["initialprice_USD"] = det_df["initialprice_USD"] / 100


det_df.rename(columns={"discount": "discount_pct"}, inplace=True)

#### "tags" column contains the steamspy tag and votes for the tag in a dict like values.
#### We will change the values to get only tag name of the dict like data.
#### The number of votes which were given for tags, are not required in this project. However, they can be analyzed separately depending on interest.

In [15]:
# Some apps do not have information about tags and instead of null the value is "[]"
det_df["tags"][det_df["tags"].str.contains("[]", regex=False)]
det_df["tags"].replace("[]", None, inplace=True)

In [16]:
# Here we remove any unnecessary characters to get just the tag names as comma separated strings
det_df["tags"] = det_df["tags"].map(
    lambda x: ", ".join(
        [i for tup in re.findall(r'"(.*?)"|\'(.*?)\':', x) for i in tup if i]
    )
    if pd.notnull(x)
    else x
)

Note: some values contain single quotes inside the string and enclosed in double qoutes, for that reason the following code didn't work: 

In [1]:
# det_df["tags"] = det_df["tags"].map(lambda x: ", ".join(re.findall(r"'(.*?)':", x)) if pd.notnull(x) else x)

#### Let's take a look at languages column, and create a column to see if the app supports English language or not.
#### Rows which do not contain "English" or contain "English, Not supported" will be "No", and the rest will be "Yes"

In [18]:
det_df.loc[:, "english"] = det_df["languages"].apply(
    lambda x: "No"
    if isinstance(x, str) and "English, Not supported" in x
    else ("Yes" if isinstance(x, str) and "English" in x.split(", ") else "No")
)

#### "rating" and "english" are the last columns of our DataFrame. Let's replace "rating" before "positive" and "negative" votes and "english" before "languages" columns

In [19]:
cols = det_df.columns.tolist()
cols.remove("rating")
cols.insert(cols.index("publisher") + 1, "rating")
det_df = det_df[cols]
det_df.dtypes

cols1 = det_df.columns.tolist()
cols1.remove("english")
cols1.insert(cols1.index("discount_pct") + 1, "english")
det_df = det_df[cols1]

#### Some rows, do not have an app name and some do not contain other relavant information such as genre, publisher or developer. These are estimated to be demo games, videos, tests and etc.
#### We will drop those rows from data because we are interested in apps, which are games and save them separately csv file.

In [23]:
# drop apps without name 
det_df[det_df["name"].isna()]
det_df2 = det_df.dropna(subset=["name"]) # 13 rows dropped

In [24]:
# drop apps if data is missing in the following columns combined
mask = det_df2[["developer", "publisher", "genre"]].isna().all(axis=1)
det_df3 = det_df2[~mask] # 81 rows dropped

#### Handling missing values
 - For columns with categorical datatype, we will fill missing values with empty string - ""
 - For columns with numeric datatype, we will fill missing values with nulls - 0

In [29]:
import warnings
warnings.filterwarnings('ignore')

In [30]:
det_df3["rating"].fillna(0, inplace=True)
det_df3["price_USD"].fillna(0, inplace=True)
det_df3["initialprice_USD"].fillna(0, inplace=True)
det_df3["discount_pct"].fillna(0, inplace=True)
det_df3["developer"] = det_df3["developer"].fillna("")
det_df3["publisher"] = det_df3["publisher"].fillna("")
det_df3["languages"] = det_df3["languages"].fillna("")
det_df3["genre"] = det_df3["genre"].fillna("")
det_df3["tags"] = det_df3["tags"].fillna("")

In [33]:
det_df3.isnull().sum().value_counts()

0    17
Name: count, dtype: int64

In [31]:
det_df3

,appid,name,developer,publisher,rating,positive,negative,avg_owners,average_forever,median_forever,price_USD,initialprice_USD,discount_pct,english,languages,genre,tags
0,570,Dota 2,Valve,Valve,82.3,1675039,360483,350000000,37327,902,0.00,0.00,0.0,Yes,"English, Bulgarian, Czech, Danish, Dutch, Finn...","Action, Free to Play, Strategy","Free to Play, MOBA, Multiplayer, Strategy, e-s..."
1,1063730,New World,Amazon Games,Amazon Games,70.2,182077,77125,75000000,4641,1946,39.99,39.99,0.0,Yes,"English, French, Italian, German, Spanish - Sp...","Action, Adventure, Massively Multiplayer, RPG","Massively Multiplayer, Open World, MMORPG, RPG..."
2,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",57.4,1269439,943280,75000000,24540,8582,0.00,0.00,0.0,Yes,"English, Korean, Simplified Chinese, French, G...","Action, Adventure, Free to Play, Massively Mul...","Survival, Shooter, Battle Royale, Multiplayer,..."
3,1172470,Apex Legends,Respawn Entertainment,Electronic Arts,80.5,568429,137990,75000000,8255,1279,0.00,0.00,0.0,Yes,"English, French, Italian, German, Spanish - Sp...","Action, Adventure, Free to Play","Free to Play, Multiplayer, Battle Royale, Shoo..."
4,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,88.8,6612258,833900,75000000,29864,6805,0.00,0.00,0.0,Yes,"English, Czech, Danish, Dutch, Finnish, French...","Action, Free to Play","FPS, Shooter, Multiplayer, Competitive, Action..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63052,848400,Simian Rising,All Caps Industries,All Caps Industries,0.0,0,0,10000,0,0,12.99,12.99,0.0,Yes,English,"Action, Adventure, Indie, Simulation, Strategy...",
63053,2479580,Prop Game,Olivette Noa,Olivette Noa,100.0,4,0,10000,0,0,1.99,1.99,0.0,Yes,"English, French","Casual, Indie","Casual, Arcade, Battle Royale, 3D Fighter, 3D ..."
63054,2105370,Mortal Fighter,Fly High Games,Fly High Games,71.4,5,2,10000,0,0,0.99,0.99,0.0,Yes,English,Action,"Martial Arts, Action, PvP, PvE, 3D Fighter, 3D..."
63055,1859220,疯狂厨房VR,703 Studio,703 Studio,0.0,0,1,10000,0,0,7.99,7.99,0.0,No,"English, Not supported, Simplified Chinese","Casual, Simulation","Casual, VR, Simulation, Funny, Stylized, Cooki..."


#### Save both DataFrames to new csv files in order to have medium steps backed up

In [ ]:
det_df.to_csv(
    r"..\SteamSpy\steam_spy_apps.csv",
    index=False,
)

det_df3.to_csv(
    r"..\SteamSpy\steam_spy_apps_cleaned.csv",
    index=False,
)

#### We did preliminary cleaning of our data. For data exploration and visualization we will continue with steam_spy_apps_cleaned.csv dataset.

### Useful information about DataFrames and csv files:
#### details_data.csv
 - Raw data downloaded from API
#### det_df
 - 6 columns dropped from raw data
 - "rating" column created from "positive" and "negative" columns
 - "english" column created from "languages" column 
 - "owners" column changed into "avg_owners" (values now average of two numbers)
 - "price", "initialprice" and "discount" columns renamed and divided by 100
 - "tags" column changed from dict like objects into comma separated strings
 - saved into file steam_spy_apps.csv
#### det_df3
 - 13 rows with missing values in "name" column dropped from det_df
 - 81 rows dropped from det_df where data was missing in the "developer", "publisher" and "genre" columns combined
 - saved into file steam_spy_apps_cleaned.csv